In [ ]:
!nohup /Users/bytedance/workplace/env/py39/bin/jupyter notebook --prot=7778 > nohup.log 2>&1 &

In [1]:
import os
import talib 
import numpy as np  
import pandas as pd 
import matplotlib as mpl
mpl.use('TkAgg')
import matplotlib.pyplot as plt 
# import mpl_finance as mpf
import matplotlib.dates as mdates
import mplfinance as mpf
import requests
import json
import datetime
from hs_udata import stock_quote_daily
from mpl_finance import candlestick_ohlc, candlestick2_ochl
import pandas_datareader as web
from datetime import datetime, timedelta
# from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.ticker as ticker
import warnings
import multiprocessing
from multiprocessing import Pool
warnings.filterwarnings('ignore')

/Users/bytedance/workplace/env/py39/lib/python3.9/site-packages/mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


In [3]:
# 日线
def get_klines(stock_code, klines_num):
    # url = 'http://33.push2his.eastmoney.com/api/qt/stock/kline/get?cb=jQuery11240262763897597486_1655023814001&secid=1.601788&ut=fa5fd1943c7b386f172d6893dbfba10b&fields1=f1%2Cf2%2Cf3%2Cf4%2Cf5%2Cf6&fields2=f51%2Cf52%2Cf53%2Cf54%2Cf55%2Cf56%2Cf57%2Cf58%2Cf59%2Cf60%2Cf61&klt=101&fqt=0&end=20500101&lmt=120&_=1655023814030'
    url1 = 'http://33.push2his.eastmoney.com/api/qt/stock/kline/get?cb=jQuery11240262763897597486_1655023814001&secid=1.'
    url2 = '&ut=fa5fd1943c7b386f172d6893dbfba10b&fields1=f1%2Cf2%2Cf3%2Cf4%2Cf5%2Cf6&fields2=f51%2Cf52%2Cf53%2Cf54%2Cf55%2Cf56%2Cf57%2Cf58%2Cf59%2Cf60%2Cf61&klt=101&fqt=0&end=20500101&lmt='
    url3 = '&_=1655023814030'
    url = url1 + str(stock_code) + url2 + str(klines_num) + url3
    headers = {
        'Accept': '*/*',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        'Cookie': 'qgqp_b_id=562d311e47faaceeb672652074c504e5; st_si=64093144335367; em_hq_fls=old; st_asi=delete; HAList=a-sh-603829-%u6D1B%u51EF%u80A1%u4EFD%2Ca-sh-601788-%u5149%u5927%u8BC1%u5238%2Ca-sz-300059-%u4E1C%u65B9%u8D22%u5BCC; st_pvi=82200143593863; st_sp=2022-06-03%2012%3A52%3A42; st_inirUrl=https%3A%2F%2Fwww.baidu.com%2Flink; st_sn=645; st_psi=20220612165023768-113200301201-7334803799',
        'Host': '33.push2his.eastmoney.com',
        'Pragma': 'no-cache',
        'Referer': 'http://quote.eastmoney.com/',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36'
    }

    resp = requests.get(url ,headers=headers)
    if resp.status_code != 200:
        print('request error: ', stock_code)
        return resp.status_code, '', '', []
    json_data = json.loads(resp.text.split('(')[1].split(')')[0])
    data = json_data['data']
    if data is None:
        return -1, '', '', ''
    code = data['code']
    name = data['name']
    klines = data['klines']
    return 200, code, name, klines

def print_klines(date, stock_code, klines, index):
    klins_list = []
    for s in klines:
        tmp = []
        splits = s.split(',')
        tmp.append(splits[0])
        for i in range(1,7):
            tmp.append(float(splits[i]))
        tmp.append(splits[-1])
        klins_list.append(tmp)
    klins_list = klins_list[-300:]
    data_pd = pd.DataFrame(klins_list, range(len(klins_list)), ['Date', 'Open', 'Close', 'High', 'Low',  'hands', 'Volume', 'ChangeRate'])
    data = data_pd
    #获取macd
    data["macd"], data["macd_signal"], data["macd_hist"] = talib.MACD(data['Close'])

    #获取10日均线和30日均线
    data["ma10"] = talib.MA(data["Close"], timeperiod=10)
    data["ma30"] = talib.MA(data["Close"], timeperiod=30)

    #获取rsi
    data["rsi"] = talib.RSI(data["Close"]) # 默认timeperiod=14
    data_pd.to_csv(date + '/data/' + str(stock_code) + '.csv', sep=',', index=False)

def request_all_code(index):
    # requests all code
#     上证A股
    url1 = 'http://49.push2.eastmoney.com/api/qt/clist/get?cb=jQuery11240743188493001389_1655043876846&pn='
    url2 = '&pz=20&po=1&np=1&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&invt=2&wbp2u=|0|0|0|web&fid=f3&fs=m:1+t:2,m:1+t:23&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f22,f11,f62,f128,f136,f115,f152&_=1655043876847'
#     # 沪深京A股
#     if city == 'sz':   
#         url1 = 'http://72.push2.eastmoney.com/api/qt/clist/get?cb=jQuery1124003788817037419645_1673843979225&pn='
#         url2 = '&pz=20&po=1&np=1&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&invt=2&wbp2u=1696305662688174|0|1|0|web&fid=f3&fs=m:0+t:6,m:0+t:80,m:1+t:2,m:1+t:23,m:0+t:81+s:2048&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f22,f11,f62,f128,f136,f115,f152&_=1673843979512'
    url = url1 + str(index) + url2
    headers = {
        'Accept': '*/*',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        'Cookie': 'qgqp_b_id=562d311e47faaceeb672652074c504e5; st_si=64093144335367; em_hq_fls=old; st_asi=delete; st_pvi=82200143593863; st_sp=2022-06-03%2012%3A52%3A42; st_inirUrl=https%3A%2F%2Fwww.baidu.com%2Flink; st_sn=762; st_psi=20220612222354561-113200301201-3254873925; HAList=a-sh-601788-%u5149%u5927%u8BC1%u5238%2Ca-sz-300059-%u4E1C%u65B9%u8D22%u5BCC',
        'Host': '49.push2.eastmoney.com',
        'Pragma': 'no-cache',
        'Referer': 'http://quote.eastmoney.com/',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36'
    }

    resp = requests.get(url ,headers=headers)
    if resp.status_code != 200:
        print('request error: ', index)
        return resp.status_code, '', []
    json_data = json.loads(resp.text.split('(')[1].split(')')[0])
    data = json_data['data']
    total = data['total']
    diff = data['diff']
    return 200, total, diff

# with open('./codes.txt', 'a+') as file_output:

# for i in range(1,111):
def request_klines_index(i, date, klines_num):
    start_idx = (i-1)*20
    status_code1, total, diff = request_all_code(i)
    if status_code1 != 200:
        print('request error', i)
    else:
        for infos in diff:
            start_idx += 1
            stock_code = infos['f12']
            name = infos['f14']
            status_code2, code, name, klines = get_klines(stock_code, klines_num)
            if code.startswith('688') or name.find('ST') != -1 or name.find('退市') != -1:
                continue
            if len(klines) <= 10:
                continue
            if status_code2 == 200:
                try:
#                     if os.path.exists(date + '/date/' + str(stock_code) + '_' + name + '.csv'):
#                         print(code + ' exist')
#                         continue
                    print_klines(date, code, klines, start_idx)
                    print(code + ' write')
#                     file_output.write(code + '\n')
                except:
                    print(str(start_idx)+' '+str(code)+' '+name, len(klines))
            else:
                print('request error', stock_code+' '+name)


In [4]:
date = '20230119'
if not os.path.exists(date + '/data'):
    os.makedirs(date + '/data')
for i in range(1,120):
    print(i)
    request_klines_index(i, date, 300)
 


1
600611 write
600536 write
603106 write
600771 write
601699 write
603383 write
603859 write
600705 write
2
601969 write
600456 write
600335 write
603728 write
600586 write
600477 write
600361 write
601528 write
3
603027 write
603590 write
605218 write
603161 write
600422 write
603520 write
603776 write
603093 write
601136 write
4
603031 write
603486 write
603586 write
600276 write
605186 write
600877 write
603203 write
600990 write
5
605398 write
603501 write
603899 write
600728 write
603701 write
600455 write
603960 write
600560 write
6
601388 write
600861 write
603721 write
603629 write
600621 write
603896 write
7
600753 write
600165 write
601990 write
601956 write
600885 write
600588 write
603229 write
600635 write
600783 write
600657 write
8
600128 write
605376 write
603528 write
600703 write
600633 write
600927 write
600318 write
9
601801 write
603588 write
600329 write
603456 write
601377 write
600745 write
600707 write
603703 write
603255 write
603882 write
601038 write
603698 

603132 write
600498 write
605122 write
600935 write
603779 write
601188 write
600397 write
603739 write
600538 write
51
600490 write
600108 write
605068 write
600655 write
600812 write
605303 write
601228 write
603585 write
603980 write
601101 write
600725 write
603113 write
600066 write
603995 write
601005 write
600704 write
52
600691 write
600148 write
603069 write
600476 write
603866 write
603559 write
603861 write
603052 write
601949 write
603020 write
600448 write
603680 write
601512 write
603890 write
601068 write
603324 write
600359 write
600819 write
603166 write
53
600158 write
600323 write
600307 write
600179 write
600618 write
600843 write
605158 write
600651 write
605111 write
600789 write
603969 write
600355 write
600749 write
600604 write
601058 write
600887 write
603789 write
54
600116 write
600511 write
600500 write
601669 write
603626 write
601766 write
600308 write
603392 write
600353 write
600824 write
600802 write
603051 write
603089 write
601279 write
600151 write


603305 write
603303 write
600019 write
603566 write
603360 write
600506 write
600694 write
603669 write
603012 write
600858 write
601606 write
90
600223 write
603938 write
600597 write
600015 write
600973 write
600060 write
600121 write
603766 write
603109 write
603600 write
601179 write
600884 write
605088 write
600809 write
603040 write
603258 write
603757 write
91
601398 write
605167 write
601177 write
600744 write
601117 write
600609 write
600876 write
603123 write
603221 write
605286 write
600795 write
601717 write
600135 write
600810 write
600048 write
92
605177 write
600315 write
603059 write
600387 write
603122 write
600267 write
601633 write
600833 write
605003 write
603176 write
605500 write
603187 write
603733 write
601933 write
603530 write
603197 write
601811 write
600956 write
603828 write
93
600733 write
600300 write
603335 write
601098 write
605366 write
600988 write
603933 write
600866 write
600730 write
605198 write
600865 write
600339 write
603587 write
603090 write


TypeError: 'NoneType' object is not subscriptable

In [5]:
# 60分钟线
def get_klines(stock_code):
    # url = 'http://33.push2his.eastmoney.com/api/qt/stock/kline/get?cb=jQuery11240262763897597486_1655023814001&secid=1.601788&ut=fa5fd1943c7b386f172d6893dbfba10b&fields1=f1%2Cf2%2Cf3%2Cf4%2Cf5%2Cf6&fields2=f51%2Cf52%2Cf53%2Cf54%2Cf55%2Cf56%2Cf57%2Cf58%2Cf59%2Cf60%2Cf61&klt=101&fqt=0&end=20500101&lmt=120&_=1655023814030'
    url1 = 'http://78.push2his.eastmoney.com/api/qt/stock/kline/get?cb=jQuery35103756490862181059_1673350047167&secid=1.'
    url2 = '&ut=fa5fd1943c7b386f172d6893dbfba10b&fields1=f1%2Cf2%2Cf3%2Cf4%2Cf5%2Cf6&fields2=f51%2Cf52%2Cf53%2Cf54%2Cf55%2Cf56%2Cf57%2Cf58%2Cf59%2Cf60%2Cf61&klt=60&fqt=1&beg=0&end=20500101&smplmt=460&lmt=1000000&_=1673350047237'
    url = url1 + str(stock_code) + url2
    headers = {
        'Accept': '*/*',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        'Cookie': 'qgqp_b_id=562d311e47faaceeb672652074c504e5; st_si=64093144335367; em_hq_fls=old; st_asi=delete; HAList=a-sh-603829-%u6D1B%u51EF%u80A1%u4EFD%2Ca-sh-601788-%u5149%u5927%u8BC1%u5238%2Ca-sz-300059-%u4E1C%u65B9%u8D22%u5BCC; st_pvi=82200143593863; st_sp=2022-06-03%2012%3A52%3A42; st_inirUrl=https%3A%2F%2Fwww.baidu.com%2Flink; st_sn=645; st_psi=20220612165023768-113200301201-7334803799',
        'Host': '33.push2his.eastmoney.com',
        'Pragma': 'no-cache',
        'Referer': 'http://quote.eastmoney.com/',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36'
    }

    resp = requests.get(url ,headers=headers)
    if resp.status_code != 200:
        print('request error: ', stock_code)
        return resp.status_code, '', '', []
    json_data = json.loads(resp.text.split('(')[1].split(')')[0])
    data = json_data['data']
    if data is None:
        return -1, '', '', ''
    code = data['code']
    name = data['name']
    klines = data['klines']
    return 200, code, name, klines

def print_klines(date, stock_code, klines, index):
    klins_list = []
    for s in klines:
        tmp = []
        splits = s.split(',')
        tmp.append(splits[0])
        for i in range(1,7):
            tmp.append(float(splits[i]))
        klins_list.append(tmp)
    klins_list = klins_list[-300:]
    data_pd = pd.DataFrame(klins_list, range(len(klins_list)), ['Date', 'Open', 'Close', 'High', 'Low',  'hands', 'Volume'])
    data = data_pd
    #获取macd
    data["macd"], data["macd_signal"], data["macd_hist"] = talib.MACD(data['Close'])

    #获取10日均线和30日均线
    data["ma10"] = talib.MA(data["Close"], timeperiod=10)
    data["ma30"] = talib.MA(data["Close"], timeperiod=30)

    #获取rsi
    data["rsi"] = talib.RSI(data["Close"]) # 默认timeperiod=14
    data_pd.to_csv(date + '/data_60/' + str(stock_code) + '.csv', sep=',', index=False)

def request_all_code(index):
    # requests all code
    url1 = 'http://49.push2.eastmoney.com/api/qt/clist/get?cb=jQuery11240743188493001389_1655043876846&pn='
    url2 = '&pz=20&po=1&np=1&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&invt=2&wbp2u=|0|0|0|web&fid=f3&fs=m:1+t:2,m:1+t:23&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f22,f11,f62,f128,f136,f115,f152&_=1655043876847'
    url = url1 + str(index) + url2
    headers = {
        'Accept': '*/*',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        'Cookie': 'qgqp_b_id=562d311e47faaceeb672652074c504e5; st_si=64093144335367; em_hq_fls=old; st_asi=delete; st_pvi=82200143593863; st_sp=2022-06-03%2012%3A52%3A42; st_inirUrl=https%3A%2F%2Fwww.baidu.com%2Flink; st_sn=762; st_psi=20220612222354561-113200301201-3254873925; HAList=a-sh-601788-%u5149%u5927%u8BC1%u5238%2Ca-sz-300059-%u4E1C%u65B9%u8D22%u5BCC',
        'Host': '49.push2.eastmoney.com',
        'Pragma': 'no-cache',
        'Referer': 'http://quote.eastmoney.com/',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36'
    }

    resp = requests.get(url ,headers=headers)
    if resp.status_code != 200:
        print('request error: ', index)
        return resp.status_code, '', []
    json_data = json.loads(resp.text.split('(')[1].split(')')[0])
    data = json_data['data']
    total = data['total']
    diff = data['diff']
    return 200, total, diff

# with open('./codes.txt', 'a+') as file_output:

# for i in range(1,111):
def request_klines_index(i, date):
    start_idx = (i-1)*20
    status_code1, total, diff = request_all_code(i)
    if status_code1 != 200:
        print('request error', i)
    else:
        for infos in diff:
            start_idx += 1
            stock_code = infos['f12']
            name = infos['f14']
            status_code2, code, name, klines = get_klines(stock_code)
            if code.startswith('688') or name.find('ST') != -1 or name.find('退市') != -1:
                continue
            if len(klines) <= 10:
                continue
            if status_code2 == 200:
                try:
#                     if os.path.exists(date + '/date/' + str(stock_code) + '_' + name + '.csv'):
#                         print(code + ' exist')
#                         continue
                    print_klines(date, code, klines, start_idx)
                    print(code + ' write')
#                     file_output.write(code + '\n')
                except:
                    print(str(start_idx)+' '+str(code)+' '+name, len(klines))
            else:
                print('request error', stock_code+' '+name)


In [6]:
# date = '20230110'
if not os.path.exists(date + '/data_60'):
    os.makedirs(date + '/data_60')
for i in range(1,120):
    print(i)
    request_klines_index(i, date)
 

1
601828 write
603803 write
603068 write
600898 write
600077 write
600771 write
603330 write
600095 write
601118 write
600433 write
2
600579 write
600633 write
600416 write
603055 write
603929 write
603800 write
600477 write
3
605377 write
600537 write
605333 write
603000 write
600460 write
605020 write
4
603267 write
600266 write
600621 write
605086 write
603236 write
605080 write
5
603303 write
601100 write
603035 write
605222 write
603029 write
603809 write
603863 write
603758 write
603136 write
6
603893 write
603093 write
603501 write
600927 write
600038 write
7
600106 write
600501 write
603986 write
600682 write
603690 write
603163 write
600592 write
601231 write
605358 write
8
600706 write
600810 write
600576 write
601208 write
600408 write
603833 write
600707 write
9
603098 write
600733 write
601567 write
605117 write
603339 write
600198 write
600520 write
600731 write
603200 write
10
603650 write
600819 write
603678 write
600958 write
605376 write
603617 write
603687 write
6036

600815 write
600798 write
600793 write
49
600798 write
600793 write
600790 write
600786 write
600768 write
600758 write
600742 write
600740 write
600727 write
600723 write
600722 write
600717 write
50
600740 write
600727 write
600723 write
600722 write
600717 write
600691 write
600664 write
51
600664 write
600632 write
600631 write
600627 write
600625 write
600622 write
600619 write
600607 write
600606 write
600590 write
52
600553 write
600535 write
600533 write
600507 write
600496 write
600490 write
600475 write
600472 write
600470 write
600458 write
600426 write
600425 write
600403 write
600393 write
600379 write
53
600358 write
600357 write
600338 write
600336 write
600317 write
600296 write
600285 write
600273 write
600270 write
600263 write
600253 write
600251 write
600227 write
600217 write
54
600217 write
600205 write
600191 write
600187 write
600159 write
600127 write
600120 write
600103 write
600102 write
55
600075 write
600068 write
600058 write
600012 write
600010 write
6000

603628 write
603983 write
603179 write
603031 write
89
605488 write
603387 write
600881 write
600854 write
600624 write
600678 write
600466 write
601022 write
600807 write
601028 write
600976 write
605155 write
601778 write
601888 write
90
601888 write
600551 write
600211 write
603327 write
600789 write
600718 write
600028 write
600060 write
600355 write
600236 write
603968 write
600600 write
603040 write
603277 write
600756 write
603377 write
600488 write
91
603217 write
601333 write
600880 write
603477 write
600693 write
600578 write
600163 write
605007 write
600750 write
603288 write
600969 write
600982 write
603843 write
603869 write
603868 write
605196 write
600725 write
603660 write
92
603600 write
603868 write
600527 write
603896 write
600617 write
603517 write
600795 write
600875 write
600871 write
600243 write
600256 write
600397 write
600569 write
600333 write
600037 write
600756 write
600833 write
93
600778 write
600193 write
603717 write
603701 write
603969 write
600718 wri

TypeError: 'NoneType' object is not subscriptable

In [175]:
# 30分钟线
def get_klines(stock_code):
    # url = 'http://33.push2his.eastmoney.com/api/qt/stock/kline/get?cb=jQuery11240262763897597486_1655023814001&secid=1.601788&ut=fa5fd1943c7b386f172d6893dbfba10b&fields1=f1%2Cf2%2Cf3%2Cf4%2Cf5%2Cf6&fields2=f51%2Cf52%2Cf53%2Cf54%2Cf55%2Cf56%2Cf57%2Cf58%2Cf59%2Cf60%2Cf61&klt=101&fqt=0&end=20500101&lmt=120&_=1655023814030'
    url1 = 'http://90.push2his.eastmoney.com/api/qt/stock/kline/get?cb=jQuery351006702935578464086_1673350768703&secid=1.'
    url2 = '&ut=fa5fd1943c7b386f172d6893dbfba10b&fields1=f1%2Cf2%2Cf3%2Cf4%2Cf5%2Cf6&fields2=f51%2Cf52%2Cf53%2Cf54%2Cf55%2Cf56%2Cf57%2Cf58%2Cf59%2Cf60%2Cf61&klt=30&fqt=1&beg=0&end=20500101&smplmt=460&lmt=1000000&_=1673350768804'
    url = url1 + str(stock_code) + url2
    headers = {
        'Accept': '*/*',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        'Cookie': 'qgqp_b_id=562d311e47faaceeb672652074c504e5; st_si=64093144335367; em_hq_fls=old; st_asi=delete; HAList=a-sh-603829-%u6D1B%u51EF%u80A1%u4EFD%2Ca-sh-601788-%u5149%u5927%u8BC1%u5238%2Ca-sz-300059-%u4E1C%u65B9%u8D22%u5BCC; st_pvi=82200143593863; st_sp=2022-06-03%2012%3A52%3A42; st_inirUrl=https%3A%2F%2Fwww.baidu.com%2Flink; st_sn=645; st_psi=20220612165023768-113200301201-7334803799',
        'Host': '33.push2his.eastmoney.com',
        'Pragma': 'no-cache',
        'Referer': 'http://quote.eastmoney.com/',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36'
    }

    resp = requests.get(url ,headers=headers)
    if resp.status_code != 200:
        print('request error: ', stock_code)
        return resp.status_code, '', '', []
    json_data = json.loads(resp.text.split('(')[1].split(')')[0])
    data = json_data['data']
    if data is None:
        return -1, '', '', ''
    code = data['code']
    name = data['name']
    klines = data['klines']
    return 200, code, name, klines

def print_klines(date, stock_code, klines, index):
    klins_list = []
    for s in klines:
        tmp = []
        splits = s.split(',')
        tmp.append(splits[0])
        for i in range(1,7):
            tmp.append(float(splits[i]))
        klins_list.append(tmp)
    klins_list = klins_list[-300:]
    data_pd = pd.DataFrame(klins_list, range(len(klins_list)), ['Date', 'Open', 'Close', 'High', 'Low',  'hands', 'Volume'])
    data = data_pd
    #获取macd
    data["macd"], data["macd_signal"], data["macd_hist"] = talib.MACD(data['Close'])

    #获取10日均线和30日均线
    data["ma10"] = talib.MA(data["Close"], timeperiod=10)
    data["ma30"] = talib.MA(data["Close"], timeperiod=30)

    #获取rsi
    data["rsi"] = talib.RSI(data["Close"]) # 默认timeperiod=14
    data_pd.to_csv(date + '/data_30/' + str(stock_code) + '.csv', sep=',', index=False)

def request_all_code(index):
    # requests all code
    url1 = 'http://49.push2.eastmoney.com/api/qt/clist/get?cb=jQuery11240743188493001389_1655043876846&pn='
    url2 = '&pz=20&po=1&np=1&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&invt=2&wbp2u=|0|0|0|web&fid=f3&fs=m:1+t:2,m:1+t:23&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f22,f11,f62,f128,f136,f115,f152&_=1655043876847'
    url = url1 + str(index) + url2
    headers = {
        'Accept': '*/*',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        'Cookie': 'qgqp_b_id=562d311e47faaceeb672652074c504e5; st_si=64093144335367; em_hq_fls=old; st_asi=delete; st_pvi=82200143593863; st_sp=2022-06-03%2012%3A52%3A42; st_inirUrl=https%3A%2F%2Fwww.baidu.com%2Flink; st_sn=762; st_psi=20220612222354561-113200301201-3254873925; HAList=a-sh-601788-%u5149%u5927%u8BC1%u5238%2Ca-sz-300059-%u4E1C%u65B9%u8D22%u5BCC',
        'Host': '49.push2.eastmoney.com',
        'Pragma': 'no-cache',
        'Referer': 'http://quote.eastmoney.com/',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36'
    }

    resp = requests.get(url ,headers=headers)
    if resp.status_code != 200:
        print('request error: ', index)
        return resp.status_code, '', []
    json_data = json.loads(resp.text.split('(')[1].split(')')[0])
    data = json_data['data']
    total = data['total']
    diff = data['diff']
    return 200, total, diff

# with open('./codes.txt', 'a+') as file_output:

# for i in range(1,111):
def request_klines_index(i, date):
    start_idx = (i-1)*20
    status_code1, total, diff = request_all_code(i)
    if status_code1 != 200:
        print('request error', i)
    else:
        for infos in diff:
            start_idx += 1
            stock_code = infos['f12']
            name = infos['f14']
            status_code2, code, name, klines = get_klines(stock_code)
            if code.startswith('688') or name.find('ST') != -1 or name.find('退市') != -1:
                continue
            if len(klines) <= 10:
                continue
            if status_code2 == 200:
                try:
#                     if os.path.exists(date + '/date/' + str(stock_code) + '_' + name + '.csv'):
#                         print(code + ' exist')
#                         continue
                    print_klines(date, code, klines, start_idx)
                    print(code + ' write')
#                     file_output.write(code + '\n')
                except:
                    print(str(start_idx)+' '+str(code)+' '+name, len(klines))
            else:
                print('request error', stock_code+' '+name)


In [176]:
date = '20230110'
if not os.path.exists(date + '/data_30'):
    os.makedirs(date + '/data_30')
for i in range(1,120):
    print(i)
    request_klines_index(i, date)
 

1
600658 write
600733 write
600936 write
603227 write
600509 write
600729 write
605289 write
603595 write
605133 write
603738 write
600537 write
603917 write
603100 write
2
600859 write
600521 write
603102 write
600785 write
600418 write
600201 write
603179 write
601127 write
3
603236 write
600446 write
603203 write
603758 write
605376 write
600861 write
603338 write
600831 write
603035 write
605009 write
4
603383 write
600152 write
600060 write
600171 write
600702 write
600584 write
605168 write
605117 write
600085 write
603170 write
605599 write
600970 write
5
603598 write
603538 write
600212 write
603186 write
601633 write
600161 write
603161 write
600006 write
600079 write
603105 write
6
603819 write
600520 write
603081 write
603508 write
600827 write
603949 write
603069 write
601137 write
603809 write
603882 write
603283 write
603290 write
603701 write
605499 write
7
601377 write
603501 write
603868 write
603356 write
603306 write
600763 write
601139 write
600887 write
603516 writ

600463 write
600468 write
603111 write
51
600377 write
600188 write
601808 write
600054 write
600906 write
603997 write
603189 write
603122 write
600825 write
601899 write
603215 write
601375 write
603968 write
601113 write
603816 write
601233 write
52
600438 write
600228 write
603380 write
601616 write
601011 write
600741 write
600556 write
603786 write
600998 write
600510 write
603456 write
600547 write
600131 write
605365 write
53
600988 write
603588 write
601006 write
600487 write
605277 write
601988 write
600585 write
603192 write
603578 write
603589 write
600797 write
603280 write
54
603599 write
600114 write
603602 write
600198 write
603191 write
601200 write
603355 write
603583 write
601916 write
601106 write
601288 write
603838 write
603360 write
600620 write
603656 write
55
603333 write
605006 write
603118 write
600110 write
600410 write
600148 write
603618 write
601177 write
603596 write
605100 write
600868 write
603221 write
56
601990 write
600908 write
600869 write
603515 

603655 write
600834 write
600262 write
600106 write
600444 write
600573 write
601880 write
600515 write
600477 write
603177 write
601116 write
605196 write
90
600622 write
603300 write
601798 write
605255 write
600784 write
603216 write
601599 write
601005 write
603915 write
600501 write
600605 write
600405 write
600689 write
600823 write
601838 write
600630 write
600778 write
91
603095 write
600738 write
600403 write
603098 write
601788 write
600758 write
600676 write
600273 write
603132 write
600595 write
601399 write
603918 write
600143 write
603379 write
603877 write
92
600865 write
603188 write
605055 write
601001 write
603959 write
603079 write
601985 write
600012 write
603439 write
601319 write
603900 write
600281 write
603168 write
689009 write
600015 write
603112 write
600895 write
605555 write
603232 write
93
601789 write
600488 write
600022 write
603848 write
603299 write
603393 write
600058 write
603167 write
603255 write
603826 write
600580 write
600968 write
605199 write


TypeError: 'NoneType' object is not subscriptable

In [ ]:
# 画图
import os
import talib 
import numpy as np  
import pandas as pd 
import matplotlib as mpl
mpl.use('TkAgg')
import matplotlib.pyplot as plt 
# import mpl_finance as mpf
import matplotlib.dates as mdates
import mplfinance as mpf
import requests
import json
import datetime
from hs_udata import stock_quote_daily
from mpl_finance import candlestick_ohlc, candlestick2_ochl
import pandas_datareader as web
from datetime import datetime, timedelta
# from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.ticker as ticker
import warnings
import multiprocessing
from multiprocessing import Pool
warnings.filterwarnings('ignore')

plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签 
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号 


def print_klines(stock_code, cur_date):
    if not os.path.exists(cur_date + '/png_error'):
        os.mkdir(cur_date + '/png_error')
    data = pd.read_csv('20230106/data/' + str(stock_code) + '.csv')
    #获取macd
    data["macd"], data["macd_signal"], data["macd_hist"] = talib.MACD(data['Close'])

    #获取10日均线和30日均线
    data["ma10"] = talib.MA(data["Close"], timeperiod=10)
    data["ma30"] = talib.MA(data["Close"], timeperiod=30)

    #获取rsi
    data["rsi"] = talib.RSI(data["Close"])
    
    fig = plt.figure()  #创建绘图区，包含四个子图
    fig.set_size_inches((20, 16))
    ax_candle = fig.add_axes((0, 0.72, 1, 0.32))   #蜡烛图子图
    ax_macd = fig.add_axes((0, 0.48, 1, 0.2), sharex=ax_candle)  #macd子图
    ax_rsi = fig.add_axes((0, 0.24, 1, 0.2), sharex=ax_candle)  #rsi子图
    ax_vol = fig.add_axes((0, 0, 1, 0.2), sharex=ax_candle)   #成交量子图
    ax_vol = fig.add_axes((0, 0, 1, 0.2))   #成交量子图

    ohlc = []   #存放行情数据，candlestick_ohlc需要传入固定格式的数据
    row_number = 0
    for date, row in data.iterrows():
        date, openp, closep, highp, lowp = row[:5]
        ohlc.append([row_number, openp, highp, lowp, closep])
        row_number = row_number+1

    date_tickers = data.Date.values #获取Date数据

    #绘制蜡烛图
    ax_candle.plot(data.index, data["ma10"], label="MA10")
    ax_candle.plot(data.index, data["ma30"], label="MA30")
    candlestick_ohlc(ax_candle, ohlc, colorup="r", colordown="g", width=0.8)
    ax_candle.xaxis.set_major_locator(ticker.MultipleLocator(6))
    ax_candle.xaxis.set_major_formatter(ticker.FixedFormatter(date_tickers)) #设置x轴刻度
    ax_candle.grid(True)
    ax_candle.set_title(str(stock_code), fontsize=20)
    ax_candle.legend()

    #绘制MACD
    ax_macd.plot(data.index, data["macd"], label="macd")
    ax_macd.bar(data.index, data["macd_hist"] * 3, label="hist")
    ax_macd.plot(data.index, data["macd_signal"], label="signal")
    ax_macd.set_title('MACD')
    ax_macd.legend()

    #绘制RSI
#     ax_rsi.set_ylabel("(%)")
    ax_rsi.plot(data.index, [70] * len(data.index), label="overbought")
    ax_rsi.plot(data.index, [30] * len(data.index), label="oversold")
    ax_rsi.plot(data.index, data["rsi"], label="rsi")
    ax_rsi.set_title('KDJ')
    ax_rsi.legend()

    #绘制成交量
    ax_vol.bar(data.index, data["Volume"] / 1000000)
    ax_vol.set_ylabel("(Million)")

    #保存图片到本地
    # fig.savefig("/Users/answer/Desktop/investment/photos/" + title + ".png", bbox_inches="tight")

    #这里个人选择不要plt.show()，因为保存图片到本地的
#     plt.show()
    plt.savefig(cur_date + '/png_error/'+str(stock_code)+'.png', bbox_inches="tight")


In [ ]:
print_klines(600935, '20230106')


In [35]:
#计算股票n天收益
def stock_dayn_profit(stock_code, date, dayn, cur_date):
#     stock_code = 601238
#     date = '2022-10-10'
#     dayn = 20
    data = pd.read_csv(cur_date + '/data/' + str(stock_code) + '.csv')
    idx = data.loc[data['Date'] == date].index[0]
    end_idx = idx + dayn
    start_price = data.loc[idx]['Close']
    end_price = data.loc[end_idx]['Close']
    price_diff = (end_price - start_price) / start_price
    return price_diff

def read_stock(stock_code, dayn, cur_date):
    data = pd.read_csv(cur_date + '/data/' + str(stock_code) + '.csv')
    indexs = data[data['rsi'] < 30].index
    if len(indexs) == 0:
        return 0,0,0,0
#     print(data[data['rsi'] < 30])
    index_max = data.index[-1]
#     print(index_max)
    all_profit = 0.0
    succ_num = 0
    fail_num = 0
    equal_num = 0
    for i in range(len(indexs)):
        if indexs[i]+1 + dayn > index_max:
            break
        date = data.loc[indexs[i]]['Date']  
        close =  data.loc[indexs[i]]['Close']
        rsi = data.loc[indexs[i]]['rsi']
        if indexs[i]-1 < 0 or indexs[i]+1 > index_max:
            break
        if data.loc[indexs[i]+1]['rsi'] > rsi and rsi < data.loc[indexs[i]-1]['rsi']:
            profit_rate = stock_dayn_profit(stock_code, data.loc[indexs[i]+1]['Date'], dayn, cur_date)
            if profit_rate > 0:
                succ_num += 1
            elif profit_rate < 0:
                fail_num += 1
            else:
                equal_num += 1
#             print(date, profit_rate)
            all_profit += profit_rate
#     return all_profit / len(indexs), succ_num, fail_num, equal_num
    return all_profit, succ_num, fail_num, equal_num

In [37]:
# 遍历文件
import os

date = '20230109'
dir = date + '/data'
files  = os.listdir(dir)
for dayn in range(10,15):
# dayn = 1
    print(dayn)
    total_profit = 0.0
    succ_num = 0
    fail_num = 0
    equal_num = 0
    for f in files:
        stock_code = f.split('.')[0]
        profit, succ, fail, equal = read_stock(stock_code, dayn, date)
        total_profit += profit
        succ_num += succ
        fail_num += fail
        equal_num += equal
    # print(total_profit / len(files), succ_num, fail_num, equal_num)
    print(total_profit, succ_num, fail_num, equal_num)

10
164.20699916932122 5034 2870 65
11
183.50383564244203 4833 2987 61
12
199.1813289500429 4835 2969 68
13
228.1734748480826 4873 2935 59
14
206.84765427983837 4708 3069 52


In [ ]:
1
25.626222884716206 4508 3516 226
2
63.48273730526064 4722 3369 157
3
85.50892530800576 4785 3348 112
4
96.57807896894603 4770 3365 109
5
122.09535659400505 4961 3197 73
6
136.2329204372769 5015 3073 78
7
132.91006135660777 4996 3084 73
8
151.22364157104462 5075 2998 78
9
146.8926412525936 5026 3034 77
10
164.20699916932122 5034 2870 65
11
183.50383564244203 4833 2987 61
12
199.1813289500429 4835 2969 68
13
228.1734748480826 4873 2935 59
14
206.84765427983837 4708 3069 52

In [ ]:
# 1天，197.5670945685812 8180 0 71
# 2天，224.04019201231011 6605 1518 127
# 3天，263.1071915403606 6347 1804 97
# 4天，286.03344344794493 6050 2113 82
# 5天，297.23904686383906 5957 2221 66
# 6天，323.24575695891747 6035 2123 73
# 7天，337.1953966304449 6049 2059 58
# 8天，
# 9天，
# 10天，348.61277047839815 5931 2153 53

In [10]:
data = pd.read_csv('20230109/data/600004.csv')

In [12]:
data

,Date,Open,Close,High,Low,hands,Volume,macd,macd_signal,macd_hist,ma10,ma30,rsi
0,2021-10-20,11.75,11.80,11.88,11.43,219376.0,255448852.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-10-21,11.80,11.43,11.98,11.42,179024.0,207929607.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-10-22,11.42,11.18,11.53,11.11,170814.0,194050055.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-10-25,10.97,10.96,11.00,10.50,225896.0,244268708.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-10-26,10.84,11.04,11.15,10.80,123416.0,136170070.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,2023-01-03,14.99,14.56,15.00,14.31,306890.0,445475572.0,0.056424,0.223287,-0.166862,15.344,15.068667,40.903327
296,2023-01-04,14.46,14.62,14.72,14.39,193977.0,282750135.0,-0.000258,0.178578,-0.178836,15.252,15.101667,41.878395
297,2023-01-05,14.65,15.01,15.05,14.54,279711.0,416376624.0,-0.013554,0.140152,-0.153705,15.194,15.149333,47.896213
298,2023-01-06,14.93,14.59,14.97,14.51,273806.0,401100585.0,-0.057320,0.100657,-0.157977,15.112,15.191000,42.761420


In [34]:
data.loc[0]

Date            2021-10-20
Open                 11.75
Close                 11.8
High                 11.88
Low                  11.43
hands             219376.0
Volume         255448852.0
macd                   NaN
macd_signal            NaN
macd_hist              NaN
ma10                   NaN
ma30                   NaN
rsi                    NaN
Name: 0, dtype: object

In [18]:
def read_stock_cur(stock_code, cur_date):
    data = pd.read_csv(cur_date + '/data/' + str(stock_code) + '.csv')
    cur_rsi = data.loc[data.index[-1]]['rsi']
    if cur_rsi < 30:
        print(stock_code, cur_rsi)

In [19]:
# 遍历文件
import os

date = '20230109'
dir = date + '/data'
files  = os.listdir(dir)
total_profit = 0.0
succ_num = 0
fail_num = 0
equal_num = 0
for f in files:
    stock_code = f.split('.')[0]
    read_stock_cur(stock_code, date)
    

601028 15.982210476251357


In [27]:
import os
import talib 
import numpy as np  
import pandas as pd 
import matplotlib as mpl
mpl.use('TkAgg')
import matplotlib.pyplot as plt 
# import mpl_finance as mpf
import matplotlib.dates as mdates
import mplfinance as mpf
import requests
import json
import datetime
from hs_udata import stock_quote_daily
from mpl_finance import candlestick_ohlc, candlestick2_ochl
import pandas_datareader as web
from datetime import datetime, timedelta
# from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.ticker as ticker
import warnings
import multiprocessing
from multiprocessing import Pool
warnings.filterwarnings('ignore')

def print_klines_span(stock_code, cur_date, buy_idx, start_idx, dayn):
    if not os.path.exists(cur_date + '/png_profit'):
        os.makedirs(cur_date + '/png_profit')
    output_path = cur_date + '/png_profit/'+str(stock_code) + '_' + str(buy_idx) +'.png'
    data = pd.read_csv(cur_date + '/data/' + str(stock_code) + '.csv')
    #获取macd
#     data["macd"], data["macd_signal"], data["macd_hist"] = talib.MACD(data['Close'])

    #获取10日均线和30日均线
#     data["ma10"] = talib.MA(data["Close"], timeperiod=10)
#     data["ma30"] = talib.MA(data["Close"], timeperiod=30)

    #获取rsi
#     data["rsi"] = talib.RSI(data["Close"])
    data = data[start_idx:min(start_idx+dayn, data.index[-1])]
    data.index = range(0,len(data.index))
    
    fig = plt.figure()  #创建绘图区，包含四个子图
    fig.set_size_inches((20, 16))
    ax_candle = fig.add_axes((0, 0.72, 1, 0.32))   #蜡烛图子图
    ax_macd = fig.add_axes((0, 0.48, 1, 0.2), sharex=ax_candle)  #macd子图
    ax_rsi = fig.add_axes((0, 0.24, 1, 0.2), sharex=ax_candle)  #rsi子图
    ax_vol = fig.add_axes((0, 0, 1, 0.2), sharex=ax_candle)   #成交量子图
    ax_vol = fig.add_axes((0, 0, 1, 0.2))   #成交量子图

    ohlc = []   #存放行情数据，candlestick_ohlc需要传入固定格式的数据
    row_number = 0
    for date, row in data.iterrows():
        date, openp, closep, highp, lowp = row[:5]
        ohlc.append([row_number, openp, highp, lowp, closep])
        row_number = row_number+1

#     date_tickers = data.Date.values #获取Date数据
    date_tickers = ['-'.join(val.split('-')[1:]) for val in data.loc[data[data.index % 4 == 0].index].Date.values] #获取Date数据
    date_tickers = np.insert(date_tickers, 0 ,['0','0'])

    #绘制蜡烛图
    ax_candle.plot(data.index, data["ma10"], label="MA10")
    ax_candle.plot(data.index, data["ma30"], label="MA30")
    candlestick_ohlc(ax_candle, ohlc, colorup="r", colordown="g", width=0.8)
    ax_candle.xaxis.set_major_locator(ticker.MultipleLocator(4))
    ax_candle.xaxis.set_major_formatter(ticker.FixedFormatter(date_tickers)) #设置x轴刻度
    ax_candle.grid(True)
    ax_candle.set_title(str(stock_code), fontsize=20)
    ax_candle.legend()

    #绘制MACD
    ax_macd.plot(data.index, data["macd"], label="macd")
    ax_macd.bar(data.index, data["macd_hist"] * 3, label="hist")
    ax_macd.plot(data.index, data["macd_signal"], label="signal")
    ax_macd.set_title('MACD')
    ax_macd.legend()

    #绘制RSI
    ax_rsi.set_ylabel("(%)")
    ax_rsi.plot(data.index, [70] * len(data.index), label="overbought")
    ax_rsi.plot(data.index, [30] * len(data.index), label="oversold")
    ax_rsi.plot(data.index, data["rsi"], label="rsi")
    ax_rsi.set_title('KDJ')
    ax_rsi.legend()

    #绘制成交量
    ax_vol.bar(data.index, data["Volume"] / 1000000)
    ax_vol.set_ylabel("(Million)")

    plt.savefig(cur_date + '/png_profit/'+str(stock_code) + '_' + str(buy_idx) +'.png', bbox_inches="tight")

In [28]:
print_klines_span(603596, '20230109', 114, 64, 40)

In [85]:
#计算股票n天收益
def stock_dayn_profit(stock_code, date, dayn, cur_date):
#     stock_code = 601238
#     date = '2022-10-10'
#     dayn = 20
    data = pd.read_csv(cur_date + '/data/' + str(stock_code) + '.csv')
    idx = data.loc[data['Date'] == date].index[0]
    end_idx = idx + dayn
    start_price = data.loc[idx]['Close']
    end_price = data.loc[end_idx]['Close']
    price_diff = (end_price - start_price) / start_price
    return price_diff

def read_stock(stock_code, dayn, cur_date):
    data = pd.read_csv(cur_date+'/data/' + str(stock_code) + '.csv')
    indexs = data[data['rsi'] < 30].index
#     print(data[data['rsi'] < 30])
    if len(indexs) == 0:
        return 0,0,0,0
#     print(data[data['rsi'] < 30])
    index_max = data.index[-1]
#     print(index_max)
    all_profit = 0.0
    succ_num = 0
    fail_num = 0
    equal_num = 0
#     print(indexs)
    for i in range(len(indexs)):
        if indexs[i] + 1 + dayn > index_max:
            break
        date = data.loc[indexs[i]]['Date']  
        close =  data.loc[indexs[i]]['Close']
        rsi = data.loc[indexs[i]]['rsi']
        if indexs[i]-1 < 0 or indexs[i]+1 > index_max:
            break
        if data.loc[indexs[i]+1]['rsi'] > rsi and rsi < data.loc[indexs[i]-1]['rsi']:
            profit_rate = stock_dayn_profit(stock_code, data.loc[indexs[i]+1]['Date'], dayn, cur_date)
#             print(stock_code, indexs[i]+1, data.loc[indexs[i]+1]['Date'], profit_rate, max(0, indexs[i]+1-50))
#             print_klines_span(stock_code, cur_date, indexs[i]+1, max(0, indexs[i]+1-50), 100)
            if profit_rate > 0:
                succ_num += 1
            elif profit_rate < 0:
                fail_num += 1
            else:
                equal_num += 1
#             print(date, profit_rate)
            all_profit += profit_rate
#     return all_profit / len(indexs), succ_num, fail_num, equal_num
    return all_profit, succ_num, fail_num, equal_num

In [86]:
print(read_stock(603596, 5, '20230109'))

(-0.08967789262618228, 0, 2, 0)


In [87]:
# 遍历文件
import os

date = '20230109'
dir = date + '/data'
files  = os.listdir(dir)
# for dayn in range(1, 20):
dayn = 5
print(dayn)
total_profit = 0.0
succ_num = 0
fail_num = 0
equal_num = 0
idx = 1
for f in files:
#     print(idx)
    idx += 1
    stock_code = f.split('.')[0]
    profit, succ, fail, equal = read_stock(stock_code, dayn, date)
    total_profit += profit
    succ_num += succ
    fail_num += fail
    equal_num += equal
# print(total_profit / len(files), succ_num, fail_num, equal_num)
print(total_profit, succ_num, fail_num, equal_num)


5
122.09535659400505 4961 3197 73


In [59]:
import os
import talib 
import numpy as np  
import pandas as pd 
import matplotlib as mpl
mpl.use('TkAgg')
import matplotlib.pyplot as plt 
# import mpl_finance as mpf
import matplotlib.dates as mdates
import mplfinance as mpf
import requests
import json
import datetime
from hs_udata import stock_quote_daily
from mpl_finance import candlestick_ohlc, candlestick2_ochl
import pandas_datareader as web
from datetime import datetime, timedelta
# from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.ticker as ticker
import warnings
import multiprocessing
from multiprocessing import Pool
warnings.filterwarnings('ignore')

def get_klines_data(stock_code, cur_date, buy_idx):
    data = pd.read_csv(cur_date + '/data/' + str(stock_code) + '.csv')
#     print(data.loc[buy_idx-1]['macd_hist'], data.loc[buy_idx]['macd_hist'])
#     return float(data.loc[buy_idx]['macd_hist']) < float(data.loc[buy_idx-1]['macd_hist'])
    flag1 = data.loc[buy_idx]['rsi'] <= data.loc[buy_idx+1]['rsi']
    flag2 = data.loc[buy_idx+1]['rsi'] <= data.loc[buy_idx+2]['rsi']
    if flag1:
        if flag2:
            return 4
        else:
            return 3
    else:
        if flag2:
            return 2
        else:
            return 1
    

In [58]:
# macd柱子还在变长不要
import os
cur_date = '20230109'
pos_true = 0
pos_true_rate = 0
pos_false = 0
pos_false_rate = 0
neg_true = 0
neg_true_rate = 0
neg_false = 0
neg_false_rate = 0
with open(os.path.join(cur_date,'profits.txt'), 'r', encoding='utf-8') as input:
    lines = input.readlines()
    for line in lines[1:10]:
        print(line.strip())
        code, idx, date, rate, start_idx = line.strip().split('\t')
        result = get_klines_data(code, cur_date, int(idx))
        print(result)
        if float(rate) > 0:
            if result:
                pos_true += 1
                pos_true_rate += float(rate)
            else:
                pos_false += 1
                pos_false_rate += float(rate)
        else:
            if result:
                neg_true += 1
                neg_true_rate += float(rate)
            else:
                neg_false +=1
                neg_false_rate += float(rate)
print(pos_true, pos_false, neg_true, neg_false)
print(pos_true_rate, pos_false_rate, neg_true_rate, neg_false_rate)

603026	121	2022/4/19	-0.245083341	71
-2.90941546797554 -2.594421500177729
False
600499	121	2022/4/19	-0.232323232	71
-0.3145899469109715 -0.2356248390148045
False
600338	120	2022/4/18	-0.230605286	70
-0.2639876768075022 -0.2194780341186568
False
600840	72	2008/6/12	-0.226094727	22
-0.2395260138706892 -0.2552627595350217
True
603186	120	2022/4/18	-0.225521669	70
-0.078422276796731 -0.0282022743480196
False
603351	120	2022/4/18	-0.210593763	70
-0.4282047287861945 -0.3739409838808367
False
603959	121	2022/4/19	-0.209083119	71
-0.1142682862862786 -0.0790484317389395
False
603192	120	2022/4/18	-0.195064233	70
-0.1371630106996295 -0.1159822930601923
False
600459	127	2022/4/19	-0.190687361	77
-0.165085513207686 -0.1593007323818487
False
0 0 1 8
0 0 -0.226094727 -1.738962004


In [76]:
# macd柱子还在变长不要
import os
cur_date = '20230109'
map1 = {}
map1[0] = {}
map1[1] = {}
map1_rate = {}
map1_rate[0] = {}
map1_rate[1] = {}
with open(os.path.join(cur_date,'profits.txt'), 'r', encoding='utf-8') as input:
    lines = input.readlines()
    for line in lines[1:]:
#         print(line.strip())
        code, idx, date, rate, start_idx = line.strip().split('\t')
        result = get_klines_data(code, cur_date, int(idx))
#         print(result)
        if float(rate) >= 0:
            if result in map1[1]:
                map1[1][result] += 1
                map1_rate[1][result] += float(rate)
            else:
                map1[1][result] = 1
                map1_rate[1][result] = float(rate)
        else:
            if result in map1[0]:
                map1[0][result] += 1
                map1_rate[0][result] += float(rate)
            else:
                map1[0][result] = 1
                map1_rate[0][result] = float(rate)
print(map1)
print(map1_rate)

{0: {1: 202, 3: 102, 2: 125, 4: 45}, 1: {4: 324, 1: 40, 2: 157, 3: 164}}
{0: {1: -11.868320447000006, 3: -3.7613372849999998, 2: -4.729638244000002, 4: -0.9447206029999999}, 1: {4: 18.661148124999986, 1: 1.053249576, 2: 7.200254830000001, 3: 6.635472608}}


In [ ]:
1: 4: 324,3: 164, 2: 157,1: 40
0: 4: 45,3: 102, 2: 125,1: 202
1: 4: 18.661148124999986,3: 6.635472608,2: 7.200254830000001,1: 1.053249576
0: 4: -0.9447206029999999, 3: -3.7613372849999998, 2: -4.729638244000002, 1: -11.868320447000006

In [64]:
(157+40)/(125+202)

0.6024464831804281

In [65]:
(324+164)/(45+102)

3.3197278911564627

## 策略升级，后连续两天跌抛

In [106]:
#计算股票n天收益
def stock_dayn_profit(stock_code, date, dayn, cur_date):
#     stock_code = 601238
#     date = '2022-10-10'
#     dayn = 20
    data = pd.read_csv(cur_date + '/data/' + str(stock_code) + '.csv')
    idx = data.loc[data['Date'] == date].index[0]
    end_idx = idx + dayn
    start_price = data.loc[idx]['Close']
    end_price = data.loc[end_idx]['Close']
    price_diff = (end_price - start_price) / start_price
    return price_diff

map1 = {}
map1[0] = {}
map1[1] = {}
map1_rate = {}
map1_rate[0] = {}
map1_rate[1] = {}

def read_stock(stock_code, dayn, cur_date, map1, map1_rate):
    data = pd.read_csv(cur_date+'/data/' + str(stock_code) + '.csv')
    indexs = data[data['rsi'] < 30].index
#     print(data[data['rsi'] < 30])
    if len(indexs) == 0:
        return 0,0,0,0
#     print(data[data['rsi'] < 30])
    index_max = data.index[-1]
#     print(index_max)
    all_profit = 0.0
    succ_num = 0
    fail_num = 0
    equal_num = 0
#     print(indexs)
    for i in range(len(indexs)):
        if indexs[i] + 1 + dayn > index_max:
            break
        date = data.loc[indexs[i]]['Date']  
        close =  data.loc[indexs[i]]['Close']
        rsi = data.loc[indexs[i]]['rsi']
        if indexs[i]-1 < 0 or indexs[i]+1 > index_max:
            break
        if data.loc[indexs[i]+1]['rsi'] > rsi and rsi < data.loc[indexs[i]-1]['rsi']:
            profit_rate = stock_dayn_profit(stock_code, data.loc[indexs[i]+1]['Date'], dayn, cur_date)
            flag = -1
            if data.loc[indexs[i]+2]['rsi'] > data.loc[indexs[i]+1]['rsi']:
                if data.loc[indexs[i]+2]['rsi'] < data.loc[indexs[i]+3]['rsi']:
                    flag = 4
                else:
                    flag = 3
#                 profit_rate = (data.loc[indexs[i]+3]['Close']-data.loc[indexs[i]+1]['Close'])/data.loc[indexs[i]+1]['Close']
            else:
                if data.loc[indexs[i]+2]['rsi'] < data.loc[indexs[i]+3]['rsi']:
                    flag = 2
                else:
                    flag = 1
#                 profit_rate = stock_dayn_profit(stock_code, data.loc[indexs[i]+1]['Date'], dayn, cur_date)
#             print(stock_code, indexs[i]+1, data.loc[indexs[i]+1]['Date'], profit_rate, max(0, indexs[i]+1-50))
            
            if profit_rate > 0:
                succ_num += 1
                if flag not in map1[1]:
                    map1[1][flag] = 1
                    map1_rate[1][flag] = profit_rate
                else:
                    map1[1][flag] += 1
                    map1_rate[1][flag] += profit_rate
            elif profit_rate < 0:
                fail_num += 1
                if flag not in map1[0]:
                    map1[0][flag] = 1
                    map1_rate[0][flag] = profit_rate
                else:
                    map1[0][flag] += 1
                    map1_rate[0][flag] += profit_rate
            else:
                equal_num += 1
#             print(date, profit_rate)
            all_profit += profit_rate
#     return all_profit / len(indexs), succ_num, fail_num, equal_num
    return all_profit, succ_num, fail_num, equal_num

In [104]:
read_stock('603026', 5, '20230109', map1, map1_rate)

(-0.060967312321722575, 6, 5, 0)

In [107]:
# 遍历文件
import os

date = '20230109'
dir = date + '/data'
files  = os.listdir(dir)
# for dayn in range(1, 20):
dayn = 5
print(dayn)
total_profit = 0.0
succ_num = 0
fail_num = 0
equal_num = 0
idx = 1
for f in files:
#     print(idx)
#     idx += 1
    stock_code = f.split('.')[0]
    profit, succ, fail, equal = read_stock(stock_code, dayn, date, map1, map1_rate)
    total_profit += profit
    succ_num += succ
    fail_num += fail
    equal_num += equal
# print(total_profit / len(files), succ_num, fail_num, equal_num)
print(total_profit, succ_num, fail_num, equal_num)
print(map1, map1_rate)

5
122.09535659400505 4961 3197 73
{0: {1: 1294, 2: 819, 3: 805, 4: 279}, 1: {2: 1174, 3: 1253, 4: 2156, 1: 378}} {0: {1: -63.83031087044469, 2: -27.558624271822087, 3: -30.146755626543253, 4: -7.318913620812207}, 1: {2: 54.55142112052911, 3: 52.30949554415443, 4: 134.08581828355221, 1: 10.003226035391387}}


In [ ]:
122.09535659400505 4961 3197 73

In [109]:
1174+2156+1253+378

4961

In [110]:
1294+819+805+279

3197

In [111]:
4961/(3197+4961)

0.6081147340034322

In [112]:
(1253+2156)/(805+279+1253+2156)

0.7587358112619631

## rsi最低点连涨两天买入

In [116]:
#计算股票n天收益
def stock_dayn_profit(stock_code, date, dayn, cur_date):
#     stock_code = 601238
#     date = '2022-10-10'
#     dayn = 20
    data = pd.read_csv(cur_date + '/data/' + str(stock_code) + '.csv')
    idx = data.loc[data['Date'] == date].index[0]
    end_idx = idx + dayn
    start_price = data.loc[idx]['Close']
    end_price = data.loc[end_idx]['Close']
    price_diff = (end_price - start_price) / start_price
    return price_diff

def read_stock(stock_code, dayn, cur_date):
    data = pd.read_csv(cur_date+'/data/' + str(stock_code) + '.csv')
    indexs = data[data['rsi'] < 30].index
#     print(data[data['rsi'] < 30])
    if len(indexs) == 0:
        return 0,0,0,0
#     print(data[data['rsi'] < 30])
    index_max = data.index[-1]
#     print(index_max)
    all_profit = 0.0
    succ_num = 0
    fail_num = 0
    equal_num = 0
#     print(indexs)
    for i in range(len(indexs)):
        if indexs[i] + 2 + dayn > index_max:
            break
        date = data.loc[indexs[i]]['Date']  
        close =  data.loc[indexs[i]]['Close']
        rsi = data.loc[indexs[i]]['rsi']
        if indexs[i]-1 < 0 or indexs[i]+1 > index_max:
            break
        if data.loc[indexs[i]+1]['rsi'] > rsi and rsi < data.loc[indexs[i]-1]['rsi'] and data.loc[indexs[i]+2]['rsi'] > data.loc[indexs[i]+1]['rsi']:
            profit_rate = stock_dayn_profit(stock_code, data.loc[indexs[i]+2]['Date'], dayn, cur_date)
#             print(stock_code, indexs[i]+1, data.loc[indexs[i]+1]['Date'], profit_rate, max(0, indexs[i]+1-50))
#             print_klines_span(stock_code, cur_date, indexs[i]+1, max(0, indexs[i]+1-50), 100)
            if profit_rate > 0:
                succ_num += 1
            elif profit_rate < 0:
                fail_num += 1
            else:
                equal_num += 1
#             print(date, profit_rate)
            all_profit += profit_rate
#     return all_profit / len(indexs), succ_num, fail_num, equal_num
    return all_profit, succ_num, fail_num, equal_num

In [117]:
print(read_stock(603596, 5, '20230109'))

(-0.14735239384198953, 0, 1, 0)


In [118]:
# 遍历文件
import os

date = '20230109'
dir = date + '/data'
files  = os.listdir(dir)
# for dayn in range(1, 20):
dayn = 5
print(dayn)
total_profit = 0.0
succ_num = 0
fail_num = 0
equal_num = 0
idx = 1
for f in files:
#     print(idx)
    idx += 1
    stock_code = f.split('.')[0]
    profit, succ, fail, equal = read_stock(stock_code, dayn, date)
    total_profit += profit
    succ_num += succ
    fail_num += fail
    equal_num += equal
# print(total_profit / len(files), succ_num, fail_num, equal_num)
print(total_profit, succ_num, fail_num, equal_num)


5
49.88217484355112 2715 1697 61


In [119]:
2715/(2715+1697)

0.6153671804170444

## 突破rsi=30策略

In [163]:
#计算股票n天收益
def stock_dayn_profit_maxmacd(stock_code, date, cur_date):
#     stock_code = 601238
#     date = '2022-10-10'
#     dayn = 20
    data = pd.read_csv(cur_date + '/data/' + str(stock_code) + '.csv')
    idx = data.loc[data['Date'] == date].index[0]
    end_idx = 1000
    for i in range(idx+1, data.index[-1]):
        if data.loc[i-1]['macd_hist'] > 0 and data.loc[i]['macd_hist'] < data.loc[i-1]['macd_hist']:
            end_idx = i
            break
#     print(idx, end_idx)
#     start_price = data.loc[idx]['Close']
#     end_price = data.loc[end_idx]['Close']
#     price_diff = (end_price - start_price) / start_price
    return end_idx

def stock_dayn_profit(stock_code, cur_date, start_idx, end_idx):
#     stock_code = 601238
#     date = '2022-10-10'
#     dayn = 20
    data = pd.read_csv(cur_date + '/data/' + str(stock_code) + '.csv')
    start_price = data.loc[start_idx]['Close']
    end_price = data.loc[end_idx]['Close']
    price_diff = (end_price - start_price) / start_price
    return price_diff

def read_stock(stock_code, cur_date):
    data = pd.read_csv(cur_date+'/data/' + str(stock_code) + '.csv')
    indexs = data[data['rsi'] < 30].index
    if len(indexs) == 0:
        return 0,0,0,0
    index_max = data.index[-1]
#     print(index_max)
    all_profit = 0.0
    succ_num = 0
    fail_num = 0
    equal_num = 0
#     print(indexs)
    for i in range(len(indexs)):
        date = data.loc[indexs[i]]['Date']  
        close =  data.loc[indexs[i]]['Close']
        rsi = data.loc[indexs[i]]['rsi']
        if indexs[i]+1 > index_max:
            break
        if data.loc[indexs[i]+1]['rsi'] >= 30:
#             print(indexs[i]+1)
            idxs = indexs[indexs > indexs[i]+1]
            idx_min = -1
            if len(idxs) != 0:
                idx_min = min(indexs[indexs > indexs[i]+1])
            end_idx = stock_dayn_profit_maxmacd(stock_code, data.loc[indexs[i]+1]['Date'], cur_date)
            flag = -1
            duration = 0
            if idx_min == -1:
                if end_idx == 1000:
                    profit_rate = stock_dayn_profit(stock_code, cur_date, indexs[i]+1, data.index[-1])
                    flag = 1
                    duration = data.index[-1]-(indexs[i]+1)
                else:
                    profit_rate = stock_dayn_profit(stock_code, cur_date, indexs[i]+1, end_idx)
                    flag = 2
                    duration = end_idx-(indexs[i]+1)
            else:
                if end_idx == 1000:
                    profit_rate = stock_dayn_profit(stock_code, cur_date, indexs[i]+1, idx_min)
                    flag = 3
                    duration = idx_min-(indexs[i]+1)
                else:
                    end_idx2 = min(idx_min, end_idx)
                    profit_rate = stock_dayn_profit(stock_code, cur_date, indexs[i]+1, end_idx2)
                    flag = 4
                    duration = end_idx2-(indexs[i]+1)
            
#             print(indexs[i]+1, data.loc[indexs[i]+1]['Date'], flag,duration, profit_rate)
#             profit_rate = stock_dayn_profit(stock_code, data.loc[indexs[i]+2]['Date'], dayn, cur_date)
            if profit_rate > 0:
                succ_num += 1
            elif profit_rate < 0:
                fail_num += 1
            else:
                equal_num += 1
#             print(date, profit_rate)
            all_profit += profit_rate
#     return all_profit / len(indexs), succ_num, fail_num, equal_num
    return all_profit, succ_num, fail_num, equal_num

In [164]:
print(read_stock(603596, '20230109'))

(-0.21963368122529697, 0, 2, 0)


In [155]:
data = pd.read_csv('20230109/data/603596.csv')

In [156]:
data.loc[113:130]

,Date,Open,Close,High,Low,hands,Volume,macd,macd_signal,macd_hist,ma10,ma30,rsi
113,2022-04-07,58.23,56.47,58.23,55.36,91094.0,514758128.0,-3.489518,-1.976572,-1.512946,66.016,71.956333,29.872776
114,2022-04-08,56.50,59.41,60.50,54.90,100051.0,586206192.0,-3.751404,-2.331538,-1.419866,64.754,71.209667,35.686260
115,2022-04-11,57.57,53.47,57.98,53.47,39214.0,214081989.0,-4.387681,-2.742767,-1.644914,63.008,70.321000,30.233014
116,2022-04-12,51.00,52.71,53.40,49.66,127111.0,652996736.0,-4.896813,-3.173576,-1.723237,61.399,69.328333,29.609570
117,2022-04-13,52.97,52.03,53.89,51.50,55127.0,289498224.0,-5.294147,-3.597690,-1.696457,59.945,68.386000,29.032692
118,2022-04-14,52.66,51.30,53.53,51.00,38977.0,202334542.0,-5.603349,-3.998822,-1.604527,58.294,67.412667,28.393157
119,2022-04-15,51.26,56.43,56.43,48.00,116582.0,616041712.0,-5.372516,-4.273561,-1.098955,57.115,66.667333,38.624870
120,2022-04-18,56.00,59.11,59.86,51.00,89183.0,515005632.0,-4.916649,-4.402178,-0.514470,56.525,66.201667,43.191631
121,2022-04-19,61.00,58.28,61.00,57.70,64806.0,380985952.0,-4.569668,-4.435676,-0.133992,55.810,65.807667,42.145714
122,2022-04-20,57.39,57.10,59.40,57.02,47401.0,275247632.0,-4.339873,-4.416516,0.076643,55.631,65.471333,40.639005


In [165]:
# 遍历文件
import os

date = '20230109'
dir = date + '/data'
files  = os.listdir(dir)
# for dayn in range(1, 20):
# dayn = 5
# print(dayn)
total_profit = 0.0
succ_num = 0
fail_num = 0
equal_num = 0
idx = 1
for f in files:
#     print(idx)
    idx += 1
    stock_code = f.split('.')[0]
    profit, succ, fail, equal = read_stock(stock_code, date)
    total_profit += profit
    succ_num += succ
    fail_num += fail
    equal_num += equal
# print(total_profit / len(files), succ_num, fail_num, equal_num)
print(total_profit, succ_num, fail_num, equal_num)


112.62669212084418 3349 3301 44
